In [1]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue

from src import data 
from src import analyzer


import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn import linear_model
import cufflinks as cf
cf.go_offline()

In [ ]:
##个股
#下午开盘
pm_13 = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/second_130000_sell_price.csv", index_col = 0)
pm_13 =pm_13[:'2021-06-18']
#收益率
# mom = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/mom1.csv", index_col = 0)

In [3]:
am_open = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price.csv", index_col = 0)
am_open = am_open['2018':'2021-06-18']

In [4]:
pm_close =pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price.csv", index_col = 0)
pm_close = pm_close['2018':'2021-06-18']

In [5]:
colum = [col for col in am_open.columns if col in list(pm_13.columns)]
len(colum)

4413

In [6]:
am_open_rv = am_open[colum]
pm_close_rv = pm_close[colum]
pm_13_rv = pm_13[colum]

In [15]:
#股市收益率
am_er =pm_13_rv/am_open_rv-1
pm_er = pm_close_rv/pm_13_rv-1

In [16]:
#上午指数收益
bmk_am = am_er.mean(axis =1)
#下午指数收益
bmk_pm = pm_er.mean(axis=1)

In [20]:
model =linear_model.LinearRegression()

In [66]:
#求apm因子值
def task(stock,q):
    i = 20
    a_er = am_er[stock]
    p_er = pm_er[stock]
    stats = np.full(len(a_er), np.nan)
    while i<len(a_er):
        if len(a_er.iloc[i-20:i].dropna())!= 20:
            i+=1
        else:
            s_am = np.array(a_er.iloc[i-20:i])
            s_pm = np.array(p_er.iloc[i-20:i])
            s_bmk_a =np.array(bmk_am.iloc[i-20:i])
            s_bmk_p =np.array(bmk_pm.iloc[i-20:i])
            test = np.concatenate((s_am,s_pm),axis = 0)
            bmk = np.concatenate((s_bmk_a,s_bmk_p),axis = 0)
            t.fit(bmk.reshape(-1,1),test)
#             res_a = test[0:20]-t.predict(bmk[0:20].reshape(-1,1))
            res_p = test[20:]-t.predict(bmk[20:].reshape(-1,1))
#             resid = res_a-res_p
            stat = np.mean(res_p)/(np.std(np.array(res_p))/np.sqrt(20))
            stats[i]=stat
            i+=1
    return q.put({stock:stats})

In [67]:
q = multiprocessing.Manager().Queue()
param = []
for stock in am_er.columns:
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =4) as pool:
    pool.starmap(task,param)

#extract
elements = []
while q.qsize():
    elements.append(q.get())

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:21: Runti

In [68]:
#将Queue转化为pd

d = pd.DataFrame(np.empty((len(am_er),am_er.shape[1]),dtype=object),columns = am_er.columns, index = am_er.index)
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    val = list(elements[i][stock])
    d[stock]=val

In [69]:
d.to_csv("apm_pm.csv")

In [55]:
#测试
#--------------

#最后一个20天的股票上午下午涨跌幅
am= am_er['000001.XSHE']
test_a = am[-22:-2].values
pm = pm_er['000001.XSHE']
test_p = pm[-22:-2].values

#40个数据组合为y值
test = np.concatenate((test_a,test_p),axis = 0)

In [56]:
#最后一个20天所有股票的涨跌幅平均作为指数数据
all_a =bmk_am[-22:-2].values
all_p = bmk_pm[-22:-2].values

In [57]:
#40个数据组合为x值
bmk = np.concatenate((all_a,all_p),axis = 0)

In [58]:
#回归模型
t.fit(bmk.reshape(-1,1),test)
#上午下午的残差
res_a = test[0:20]-t.predict(bmk[0:20].reshape(-1,1))
res_p = test[20:]-t.predict(bmk[20:].reshape(-1,1))

#残差求差
diff_r = res_a-res_p
#差值求统计量（标准化）
stat_i =np.mean(diff_r)*np.sqrt(20)/np.std(diff_r)

In [62]:
df = d.shift(1)